# **Reti Neurali** 

## **Esercizio 1: Download e pre-processamento dei dati.**

Scaricare e pre-processare i dati per il successivo addestramento del modello. 

Il dataset che utilizzeremo sarà CIFAR10, scaricabile dalla libreria `tensorflow.keras.datasets`

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Download dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.ravel()
y_test = y_test.ravel()

# Stampare le shape dei dati
print(f"Training set: {x_train.shape}, {y_train.shape}")
print(f"Test set: {x_test.shape}, {y_test.shape}")

2025-05-27 08:08:00.722363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748333281.044057      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748333281.128884      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Training set: (50000, 32, 32, 3), (50000,)
Test set: (10000, 32, 32, 3), (10000,)


In [2]:
from sklearn.preprocessing import StandardScaler

# Pre-processamento dei dati

# svolgimento...

x_train=x_train.reshape(x_train.shape[0], -1)
x_test=x_test.reshape(x_test.shape[0], -1)

scaler = StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

## **Esercizio 2: Creare modello MLP**

Per creare il modello MLP utilizziamo l' oggetto `MLPClassifier` dal modulo `sklearn.neural_networks`. Questo è un oggetto molto complesso che prevede la possibilità di specificare tanti parametri, permettendoci una personalizzazione molto dettagliata. Vediamo di seguito gli argomenti principali:

- `hidden_layer_sizes`: rappresenta la struttura dell' MLP, sotto forma di una tupla. La tupla deve essere composta da numeri interi, ogni numero indica il numero di neuroni presenti nel rispettivo layer.

Esempio: 

`hidden_layer_sizes` = `(100)`

creerà un solo layer con 100 neuroni

`hidden_layer_sizes` = `(100, 50)`

creerà due layer, il primo con 100 neuroni, il secondo invece con 50.

- `max_iter`: massimo numero di iterazioni per raggiungere la convergenza. 

- `activation`: indica quale funzione di attivazione utilizzare, valori possibili sono `'relu'`, `'logistic'`, `'tanh'` and `'identity'`.

- `solver`: indica quale algoritmo di ottimizzazione utilizzare, valori possibili sono `'adam'`, `'sgd'` and `'lbfgs'`.

- `learning_rate_init`: valore iniziale del learning rate.

- `verbose`: valore booleano che, se impostato su `True`, stampa l' output di ogni iterazione di training. Molto utile per monitorare il training.

- `random_state`: fissa il seed della randomizzazione.


Per iniziare creiamo un MLP molto basilare, alleniamolo e testiamone le performance. Come parametri utilizzeremo:

- `hidden_layer_sizes` = `(100)`

- `max_iter` = `20`

- `random_state` = `42`

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP 

# svolgimento...
mlp=MLPClassifier(hidden_layer_sizes = (100), max_iter = 20, random_state = 42)

# Allenare MLP

# svolgimento...
mlp.fit(x_train, y_train)

# Valutare MLP

# svolgimento...
predictions = mlp.predict(x_test)

# Stampare l' accuratezza

# svolgimento...
accuracy_test = accuracy_score(y_test, predictions)
print(f"Accuracy set: {accuracy_test}")

Accuracy set: 0.5057


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## **Esercizio 2.1: Aumentiamo i parametri del nostro modello**

Proviamo adesso ad aumentare i dettagli del nostro modello, modificando o aggiungendo i parametri sopra specificati. 

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP con più strati e altre specifiche

# svolgimento...
mlp=MLPClassifier(hidden_layer_sizes = (100, 50), max_iter = 80, random_state = 42, verbose=True, activation='logistic', solver='lbfgs')

mlp.fit(x_train, y_train)

predictions = mlp.predict(x_test)

accuracy_test = accuracy_score(y_test, predictions)

print(f"Accuracy set: {accuracy_test}")

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy set: 0.4554


## **Esercizio 3: Implementare manualmente l' algoritmo di early stopping.**

L' algoritmo di early stopping ci permette di terminare anticipatamente l' allenamento di un modello nel caso in cui questo raggiunga la convergenza. Supponiamo infatti che il nostro modello raggiunga un certo livello di accuratezza e che non riesca a migliorare oltre quel livello. Questo significa che il modello, da quel momento in poi, non sta più apprendendo nuove informazioni, per cui le successive iterazioni sono superflue, ed inoltre rischiano di essere dannose, spingendo il modello verso l' overfitting. 

L' early stopping verifica ad ogni iterazione che l' accuratezza del modello sia incrementata di una certa tolleranza. Se questa tolleranza non viene superata per un certo numero di epoche, allora possiamo decidere di stoppare l' allenamento in quanto il modello ha raggiunto la convergenza.

**N.B: per applicare early stopping è necessario specificare i seguenti parametri dell' MLP:**

- `warm_start`=`True` in modo che il training proceda dallo stato attuale del modello e non dall' inizializzazione.

- `max_iter`=`1` in modo che il modello venga allenato per una sola epoca. Per l' early stopping infatti dovremo gestire manualmente il numero di iterazioni.

In [6]:
train_fraction = 0.8  
val_fraction = 0.2

num_train = int(train_fraction * x_train.shape[0])
x_train_1 = x_train[:num_train]
y_train_1 = y_train[:num_train]

x_val = x_train[num_train:]
y_val = y_train[num_train:]

print(f"Training set: {x_train_1.shape}, {y_train_1.shape}")
print(f"Validation set: {x_val.shape}, {y_val.shape}")

Training set: (40000, 3072), (40000,)
Validation set: (10000, 3072), (10000,)


In [8]:
import numpy as np
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score 


n_total_epochs = 100  
patience = 10         
tolerance = 1e-4      

best_val_accuracy = 0.0
epochs_without_improvement = 0

In [9]:
mlp=MLPClassifier(hidden_layer_sizes = (100, 50), max_iter = 1, random_state = 42, verbose=True, warm_start=True )

for e in range(n_total_epochs):
    mlp.fit(x_train, y_train)
    
    predictions = mlp.predict(x_val)
    accuracy_val = accuracy_score(y_val, predictions)

    print(f"Epoca: {e}, Accuracy: {accuracy_val}")

    if accuracy_val > best_val_accuracy + tolerance:
        best_val_accuracy=accuracy_val
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {e+1} (no improvement for {patience} epochs)")
        break


test_predictions = mlp.predict(x_test)
test_accuracy= accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy}")

Iteration 1, loss = 1.75953688
Epoca: 0, Accuracy: 0.4552


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 2, loss = 1.56217223
Epoca: 1, Accuracy: 0.4887
Iteration 3, loss = 1.44992466
Epoca: 2, Accuracy: 0.511
Iteration 4, loss = 1.36972870
Epoca: 3, Accuracy: 0.5352
Iteration 5, loss = 1.30807250
Epoca: 4, Accuracy: 0.5496
Iteration 6, loss = 1.25765208
Epoca: 5, Accuracy: 0.5643
Iteration 7, loss = 1.21291055
Epoca: 6, Accuracy: 0.5813
Iteration 8, loss = 1.17317639
Epoca: 7, Accuracy: 0.5871
Iteration 9, loss = 1.14145858
Epoca: 8, Accuracy: 0.5978
Iteration 10, loss = 1.11246592
Epoca: 9, Accuracy: 0.6039
Iteration 11, loss = 1.08802703
Epoca: 10, Accuracy: 0.6154
Iteration 12, loss = 1.06296634
Epoca: 11, Accuracy: 0.6189
Iteration 13, loss = 1.03885621
Epoca: 12, Accuracy: 0.6248
Iteration 14, loss = 1.01714102
Epoca: 13, Accuracy: 0.6301
Iteration 15, loss = 0.99748605
Epoca: 14, Accuracy: 0.6345
Iteration 16, loss = 0.97878105
Epoca: 15, Accuracy: 0.6383
Iteration 17, loss = 0.95797839
Epoca: 16, Accuracy: 0.6389
Iteration 18, loss = 0.93786055
Epoca: 17, Accuracy: 0.648